# Team 3 - Merchant Category Recommendation
## Feature Engineering

### Team 3
- Vinicio De Sola
- Kevin Hanna
- Pri Nonis
- Bradley Nott


Here we add engineer new features and write the files out to:
    - input/engineered_train.pkl
    - input/engineered_test.pkl

In [1]:
import numpy               as np
import matplotlib.pyplot   as plt
import pandas              as pd
import seaborn             as sb

from datetime              import timedelta, datetime

from sklearn               import metrics
from sklearn.decomposition import PCA
from sklearn.cluster       import KMeans
from sklearn.mixture       import GaussianMixture
from sklearn.linear_model  import LinearRegression

from matplotlib.colors     import LogNorm

from IPython.display       import HTML, Markdown

from sklearn.metrics       import mean_squared_error

%matplotlib inline

np.random.seed(0)

import os
print(os.listdir("./input"))

['historical_transactions.csv', '.ipynb_checkpoints', 'new_merchant_transactions.csv', 'engineered_test.pkl', '.~lock.Data_Dictionary.xlsx#', 'merchants.csv', 'sample_submission.csv', 'foo.csv', 'test.csv', 'Data_Dictionary.xlsx', 'engineered_train.pkl', '.gitignore', 'train.csv']


## Read in all the data files

In [2]:
train        = pd.read_csv('input/train.csv', parse_dates=['first_active_month']).set_index('card_id')
test         = pd.read_csv('input/test.csv',  parse_dates=['first_active_month']).set_index('card_id')
merchants    = pd.read_csv('input/merchants.csv').set_index('merchant_id')
transactions_old = pd.read_csv('input/historical_transactions.csv',   parse_dates = ['purchase_date'])
transactions_new = pd.read_csv('input/new_merchant_transactions.csv', parse_dates = ['purchase_date'])

## Data Clean Up

In [3]:
# Row is missing value for first_active_month
print(test.loc['C_ID_c27b4f80f7', :])
test = test.drop(index='C_ID_c27b4f80f7')

first_active_month    NaT
feature_1               5
feature_2               2
feature_3               1
Name: C_ID_c27b4f80f7, dtype: object


# Model
## Baseline

Our baseline model uses the data provided in the "train" dataset. 

In [4]:
def calculate_baseline():
    training = train[['feature_1', 'feature_2', 'feature_3']]
    labels = train[['target']]

    data_len = len(training)
    mask = np.random.rand(data_len) < 0.8

    train_data = training.values[mask]
    train_labels = labels.values.flatten()[mask]
    test_data = training.values[~mask]
    test_labels = labels.values.flatten()[~mask]

    X = train_data
    y = train_labels

    X_test = test_data
    y_test = test_labels

    reg = LinearRegression()
    reg.fit(X, y)

    y_pred_reg = reg.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_reg)

    y_mean = train_labels.mean()
    y_test_mean = test_labels.mean()

    print("After fitting Linear Regression, the baseline Mean Squared Error is: {:0.3f}".format(mse))
    print("The mean of our training label is {:0.3f} and for testing {:0.3f}".format(y_mean, y_test_mean))
    
    return mse

baseline_mse = calculate_baseline()

After fitting Linear Regression, the baseline Mean Squared Error is: 14.269
The mean of our training label is -0.394 and for testing -0.393


# Feature Engineering

## Transform train first_active_month to numeric
The first_active_month is a date the shopper first made a purchase using Elo.  Converting this to days would make it more useful for modelling. 

In [5]:
def first_active_month_to_numeric():
    global train, test
    latest_date = max(train['first_active_month'].max(), test['first_active_month'].max()) + timedelta(days=1)
    train['first_active_delta_days'] = ((latest_date - train['first_active_month']).dt.days)
    test['first_active_delta_days']  = ((latest_date - test['first_active_month']).dt.days)

first_active_month_to_numeric()

# we no longer need first active month
del train['first_active_month']

# display(train.head())
# display(test.head())

In [6]:
display(merchants.columns)

Index(['merchant_group_id', 'merchant_category_id', 'subsector_id',
       'numerical_1', 'numerical_2', 'category_1', 'most_recent_sales_range',
       'most_recent_purchases_range', 'avg_sales_lag3', 'avg_purchases_lag3',
       'active_months_lag3', 'avg_sales_lag6', 'avg_purchases_lag6',
       'active_months_lag6', 'avg_sales_lag12', 'avg_purchases_lag12',
       'active_months_lag12', 'category_4', 'city_id', 'state_id',
       'category_2'],
      dtype='object')

## Binarize 'Y' and 'N' category
Convert to binary so they can be aggregated later. 

In [7]:
transactions_old['category_1'] = transactions_old.category_1.eq('Y').mul(1)
transactions_new['category_1'] = transactions_new.category_1.eq('Y').mul(1)

transactions_old['authorized_flag'] = transactions_old.authorized_flag.eq('Y').mul(1)
transactions_new['authorized_flag'] = transactions_new.authorized_flag.eq('Y').mul(1)

merchants['category_1'] = merchants.category_1.eq('Y').mul(1)
merchants['category_4'] = merchants.category_4.eq('Y').mul(1)


## Create some dummy variables

In [8]:
train = pd.get_dummies(train, columns=['feature_1', 'feature_2'])
test = pd.get_dummies(test, columns=['feature_1', 'feature_2'])
transactions_old = pd.get_dummies(transactions_old, columns=['category_2', 'category_3'])
transactions_new = pd.get_dummies(transactions_new, columns=['category_2', 'category_3'])
merchants = pd.get_dummies(merchants, columns=['category_2', 'most_recent_sales_range', 'most_recent_purchases_range'])

# display(train.head())
# display(test.head())
# display(transactions_old.head())
# display(transactions_new.head())


## Convert dates to numeric


In [9]:
latest_transaction = transactions_old['purchase_date'].max()

transactions_old['month_diff'] = ((latest_transaction - transactions_old['purchase_date']))//np.timedelta64(1, 'M')
transactions_old['month_diff'] += transactions_old['month_lag']

transactions_new['month_diff'] = ((latest_transaction - transactions_new['purchase_date']))//np.timedelta64(1, 'M')
transactions_new['month_diff'] += transactions_new['month_lag']


## Join Merchant Data with Transactions

In [10]:
del merchants['merchant_group_id']
del merchants['merchant_category_id']
del merchants['subsector_id']

# So we don't create duplicates from the transactions
merchants = merchants.add_suffix('_merchants')

transactions_old = transactions_old.merge(right=merchants, how='left', on=['merchant_id'])
transactions_new = transactions_new.merge(right=merchants, how='left', on=['merchant_id'])

## Aggregate New and Old Transactions with Train on `card_id` 
see: https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

In [11]:
def aggregate_transaction_data():
    global train, test
    
    # These aggregations will be performed on both new and old
    aggregation = {
#         'purchase_amount': ['sum', 'mean', 'median', 'max', 'min', 'std'],
        'purchase_amount': ['sum', 'mean', 'median', 'std'],
        'subsector_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'merchant_id': ['nunique'],
        'installments': ['mean'],
        'city_id': ['nunique'],
        'state_id': ['nunique'], 
        'category_1': ['sum'],
        'category_2_1.0': ['sum'],
        'category_2_2.0': ['sum'],
        'category_2_3.0': ['sum'],
        'category_2_4.0': ['sum'],
        'category_2_5.0': ['sum'],
        'category_3_A': ['sum'],
        'category_3_B': ['sum'],
        'category_3_C': ['sum'],
        'authorized_flag': ['sum'],
        'month_diff': ['mean', 'min', 'max', 'std'],
        
        # Merchants
        'numerical_1_merchants': ['sum', 'mean'],
        'numerical_2_merchants': ['sum', 'mean'],
        'category_1_merchants': ['sum'],
        'category_4_merchants': ['sum'],
        'avg_sales_lag3_merchants': ['mean'],
        'avg_purchases_lag3_merchants': ['mean'],
        'active_months_lag3_merchants': ['mean'],
        'avg_sales_lag6_merchants': ['mean'],
        'avg_purchases_lag6_merchants': ['mean'],
        'active_months_lag6_merchants': ['mean'],
        'avg_sales_lag12_merchants': ['mean'],
        'avg_purchases_lag12_merchants': ['mean'],
        'active_months_lag12_merchants': ['mean'],
        'category_2_1.0_merchants': ['sum'],
        'category_2_2.0_merchants': ['sum'],
        'category_2_3.0_merchants': ['sum'],
        'category_2_4.0_merchants': ['sum'],
        'category_2_5.0_merchants': ['sum'],
        'most_recent_sales_range_A_merchants': ['sum'],
        'most_recent_sales_range_B_merchants': ['sum'],
        'most_recent_sales_range_C_merchants': ['sum'],
        'most_recent_sales_range_D_merchants': ['sum'],
        'most_recent_sales_range_E_merchants': ['sum'],
        'most_recent_purchases_range_A_merchants': ['sum'],
        'most_recent_purchases_range_B_merchants': ['sum'],
        'most_recent_purchases_range_C_merchants': ['sum'],
        'most_recent_purchases_range_D_merchants': ['sum'],
        'most_recent_purchases_range_E_merchants': ['sum']
        
    }
    
    # Do aggregations on aggregated_old and join with training and test data
    aggregated_old = transactions_old.groupby(['card_id']).agg(aggregation)
    aggregated_old.columns = ['old_' + '_'.join(col).strip() for col in aggregated_old.columns.values]
    train = train.merge(right=aggregated_old, how='left', on='card_id')
    test = test.merge(right=aggregated_old, how='left', on='card_id')

    # Do aggregations on aggregated_new and join with training and test data
    aggregated_new = transactions_new.groupby(['card_id']).agg(aggregation)
    aggregated_new.columns = ['new_' + '_'.join(col).strip() for col in aggregated_new.columns.values]
    train = train.merge(right=aggregated_new, how='left', on='card_id')
    test = test.merge(right=aggregated_new, how='left', on='card_id')
    
    # Joining the transaction_new with train created some empty cells
    # Fill them with zeros
    train = train.fillna(0)
    test = test.fillna(0)

    
aggregate_transaction_data()
# display(train.head())
# display(test.head())

## A view of our new training data dataframe

In [12]:
# Put the target at the begining
train = train[['target'] + [c for c in train if c not in ['target']]]

display(train.head())
display(test.head())

,target,feature_3,first_active_delta_days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,new_most_recent_sales_range_A_merchants_sum,new_most_recent_sales_range_B_merchants_sum,new_most_recent_sales_range_C_merchants_sum,new_most_recent_sales_range_D_merchants_sum,new_most_recent_sales_range_E_merchants_sum,new_most_recent_purchases_range_A_merchants_sum,new_most_recent_purchases_range_B_merchants_sum,new_most_recent_purchases_range_C_merchants_sum,new_most_recent_purchases_range_D_merchants_sum,new_most_recent_purchases_range_E_merchants_sum
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.820283,1,246,0,0,0,0,1,0,1,...,7.0,1.0,7.0,5.0,3.0,7.0,1.0,5.0,7.0,3.0
C_ID_3d0044924f,0.392913,0,397,0,0,0,1,0,1,0,...,1.0,2.0,0.0,3.0,0.0,1.0,2.0,3.0,0.0,0.0
C_ID_d639edf6cd,0.688056,0,550,0,1,0,0,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
C_ID_186d6a6901,0.142495,0,154,0,0,0,1,0,0,0,...,0.0,1.0,1.0,5.0,0.0,0.0,1.0,2.0,4.0,0.0
C_ID_cdbd2c0db2,-0.159749,0,93,1,0,0,0,0,0,0,...,7.0,3.0,10.0,13.0,3.0,7.0,4.0,9.0,10.0,6.0


,first_active_month,feature_3,first_active_delta_days,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,...,new_most_recent_sales_range_A_merchants_sum,new_most_recent_sales_range_B_merchants_sum,new_most_recent_sales_range_C_merchants_sum,new_most_recent_sales_range_D_merchants_sum,new_most_recent_sales_range_E_merchants_sum,new_most_recent_purchases_range_A_merchants_sum,new_most_recent_purchases_range_B_merchants_sum,new_most_recent_purchases_range_C_merchants_sum,new_most_recent_purchases_range_D_merchants_sum,new_most_recent_purchases_range_E_merchants_sum
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_0ab67a22ab,2017-04-01,1,307,0,0,1,0,0,0,0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0
C_ID_130fd0cbdd,2017-01-01,0,397,0,1,0,0,0,0,0,...,2.0,0.0,3.0,1.0,4.0,2.0,0.0,3.0,2.0,3.0
C_ID_b709037bc5,2017-08-01,1,185,0,0,0,0,1,1,0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
C_ID_d27d835a9f,2017-12-01,0,63,0,1,0,0,0,1,0,...,5.0,4.0,1.0,0.0,0.0,6.0,1.0,3.0,0.0,0.0
C_ID_2b5e3df5c2,2015-12-01,1,794,0,0,0,0,1,1,0,...,0.0,1.0,1.0,3.0,1.0,0.0,1.0,1.0,2.0,2.0


## Write out the new File

In [13]:
# train.to_csv('./input/engineered_train.csv')
# test.to_csv('./input/engineered_test.csv')
train.to_pickle('./input/engineered_train.pkl')
test.to_pickle('./input/engineered_test.pkl')